[View in Colaboratory](https://colab.research.google.com/github/politicasydatos/curso_intensivo_estadistica/blob/master/Demo_encuesta_de_percepci%C3%B3n.ipynb)

# Demo de encuesta de percepción (simulación)

Norman Simón Rodríguez, MSc.
nsrodriguezc@unal.edu.co

En este cuaderno vamos a tratar de ilustrar las limitaciones de las encuestas de percepción para la evaluación de intervenciones sociales, desde un punto de vista técnico.

Vamos a usar un ejemplo de encuesta de percepción muy popular: la encuesta de intención de voto. Tenemos dos candidatos: Iván Duque y Gustavo Petro, y las opciones del voto en blanco y la abstención.

## La población

En Colombia hay 36'025.318 de personas habilitadas para votar. Si les preguntáramos a todas ellas, una por una, su intención de voto, estaríamos haciendo un **censo**. El problema es que sería imposible por razones económicas y logísticas, así que lo que hacemos es implementar una **encuesta**, que le pregunta la intención de voto a una muestra de ciudadanos, no a toda la población.

En la vida real no conocemos las características de la población, sino que nos tenemos que conformar con información recolectada en una muestra. Para efectos de la simulación, supongamos lo siguiente:

* La población son 36'025.318 de personas.
* De estas, el 50% se abstendrá, el 10% votará en blanco, el 20% votará por Iván Duque y el 20% votará por Gustavo Petro.
* Por ahora, todas las personas tienen exactamente la misma probabilidad de no responder la encuesta. O sea, todas las personas sienten el mismo nivel de felicidad al momento de responder encuestas políticas. Digamos que esa probabilidad es 50%.

**Nota:** Estos datos son simulados, no representan datos reales ni intentan replicar o evaluar ninguna encuesta de percepción en la vida real.

## La muestra

Tomemos una muestra de 8065 personas. Este tamaño de muestra lo copiamos de una [reciente encuesta de percepción](http://www.wradio.com.co/noticias/actualidad/en-encuesta-guarumo-ivan-duque-tiene-525-y-gustavo-petro-tiene-36-en-intencion-de-voto/20180610/nota/3760163.aspx) (junio de 2018). La muestra corresponde al 0.022% de la población. La encuesta en mención usó un diseño de muestreo estratificado, pero nosotros vamos a usar uno no estratificado en nuestra simulación para no complicar demasiado la explicación.

## El código

Ahora vamos a hacer nuestra simulación con un poco de código de Python.

Primero cargamos los módulos que necesitamos para hacer el procesamiento de los datos.

In [0]:
#@title
import numpy as np
import pandas as pd
import matplotlib as mplot
import scipy.stats as stats

Ahora definimos los parámetros.

In [0]:
censo_electoral = 36025318
intencion_voto = [0.5, 0.1, 0.2, 0.2] # [ ABS, BLA, DUQ, PET ]
probabilidad_respuesta = [0.5, 0.5, 0.5, 0.5 ] # [ ABS, BLA, DUQ, PET ]

Ahora creamos unas funciones que nos permitirán crear esta población todas las veces que queramos.

In [0]:
def crear_votantes(censo_electoral, intencion_voto_esp, probabilidad_respuesta_esp, abrev):
    tamano = int(censo_electoral * intencion_voto_esp)
    responderia = np.random.binomial(1, probabilidad_respuesta_esp, tamano)
    votantes = np.full((tamano, 2), abrev)
    votantes[:,1] = responderia
    return votantes


def crear_poblacion(censo_electoral, intencion_voto, probabilidad_respuesta):
    censo_electoral = float(censo_electoral)  
    ABS = crear_votantes(censo_electoral, intencion_voto[0], probabilidad_respuesta[0], "ABS")
    BLA = crear_votantes(censo_electoral, intencion_voto[1], probabilidad_respuesta[1], "BLA")
    DUQ = crear_votantes(censo_electoral, intencion_voto[2], probabilidad_respuesta[2], "DUQ")
    PET = crear_votantes(censo_electoral, intencion_voto[3], probabilidad_respuesta[3], "PET")  
    poblacion = np.vstack((ABS, BLA, DUQ, PET))
    poblacion = pd.DataFrame(poblacion)
    poblacion.columns = ['votaria_por', 'responderia_encuesta']
    poblacion['responderia_encuesta'] = pd.to_numeric(poblacion['responderia_encuesta'])
    print("Acabo de crear una población de {} votantes.".format(str(len(poblacion))))
    return poblacion

Y ahora creamos nuestra población de 36 millones de votantes:

In [0]:
poblacion = crear_poblacion(censo_electoral, intencion_voto, probabilidad_respuesta)

print(poblacion)

Ahora vamos a explorar esta población, para ver si nos quedó bien creada. Contemos cuántos votantes habría para cada opción en la población.

In [0]:
print("Frecuencias absolutas")
print(poblacion.groupby('votaria_por')['responderia_encuesta'].count()*100)

print("\nFrecuencias relativas")
print(poblacion.groupby('votaria_por')['responderia_encuesta'].count()*100\
/len(poblacion))

Y ahora contemos cuántos responderían la encuesta de percepción si se les preguntara.

In [0]:
print("Frecuencias absolutas")
print(poblacion.groupby('votaria_por')['responderia_encuesta'].sum())

print("\nFrecuencias relativas (en relación con la población entera)")
print(poblacion.groupby('votaria_por')['responderia_encuesta'].sum()/len(poblacion))

Ahora vamos a hacer un muestreo aleatorio. El tamaño de la muestra será 8065 personas. Si nos queda bien el muestreo, el panorama electoral de la muestra reflejará bien el panorama electoral de la población.

In [0]:
tamano_muestra = 8065
muestra = poblacion.sample(tamano_muestra)
muestra.head(10)

Ahora hagamos la encuesta. Algunos responden, algunos no responden la encuesta.

In [0]:
muestra_respondieron = muestra[muestra['responderia_encuesta'] == 1]
len(muestra_respondieron)

Entre estos que respondieron, ¿cómo quedaron las preferencias electorales?

In [0]:
muestra_respondieron.groupby('votaria_por')['responderia_encuesta'].count()*100\
/len(muestra_respondieron)

Por lo visto, la encuesta refleja bastante bien las preferencias electorales de la población. Con un margen de error de ±5 puntos porcentuales, esta encuesta declararía un empate.

## Añadiendo una complicación

Esta encuesta supone que la probabilidad de respuesta es la misma para los votantes de todos los candidatos. ¿Qué pasaría si cambiáramos un poco la probabilidad de respuesta?

In [0]:
probabilidad_respuesta = [0.1, 0.5, 0.3, 0.7 ] # [ ABS, BLA, DUQ, PET ]
poblacion = crear_poblacion(censo_electoral, intencion_voto, probabilidad_respuesta)
muestra = poblacion.sample(tamano_muestra)
muestra_respondieron = muestra[muestra['responderia_encuesta'] == 1]
muestra_respondieron.groupby('votaria_por')['responderia_encuesta'].count()*100\
/len(muestra_respondieron)

Con un margen de error de ±5 puntos porcentuales, ¿se declararía un empate? ¿Quién sería el ganador?

In [0]:
# Liberemos memoria
del poblacion
del muestra
del muestra_respondieron

## Aplicación en la evaluación de intervenciones

Supongamos que queremos evaluar una intervención social, y para ello vamos a utilizar una encuesta de percepción. Primero, definamos el escenario. Se dan unas becas para unos cursos de capacitación en competencias técnicas laborales en una población de interés. La encuesta de percepción busca determinar la satisfacción de las personas frente al programa.

Si la persona decide contestar la encuesta, las opciones son: Satisfecho, no satisfecho.

Si la persona decide no contestar la encuesta, se marca como NS/NR.

Digamos que la población son 5 mil personas, y la muestra de la encuesta será 250 personas (el 5%). Supongamos que en la población las preferencias se reparten así:

* Satisfechos: 30%
* No satisfechos: 70%

Supongamos que la probabilidad de respuesta es la misma tanto entre los satisfechos como los no satisfechos (50%).

In [0]:
tamano_poblacion = 5000
satisfechos = crear_votantes(tamano_poblacion, 0.3, 0.5, "SATIS")
no_satisfechos = crear_votantes(tamano_poblacion, 0.7, 0.5, "NOSAT")
poblacion = np.vstack((satisfechos, no_satisfechos))
poblacion = pd.DataFrame(poblacion)
poblacion.columns = ['opinion', 'responderia_encuesta']
poblacion['responderia_encuesta'] = pd.to_numeric(poblacion['responderia_encuesta'])

Hagamos el muestreo y la encuesta:

In [0]:
tamano_muestra = 400
muestra = poblacion.sample(tamano_muestra)
muestra_respondieron = muestra[muestra['responderia_encuesta'] == 1]
muestra_respondieron.groupby('opinion')['responderia_encuesta'].count()*100\
/len(muestra_respondieron)

Esta encuesta parece reflejar razonablemente bien las preferencias de la población. Pero ¿qué pasa si introducimos sesgo de selección?

In [0]:
satisfechos = crear_votantes(tamano_poblacion, 0.3, 0.75, "SATIS")
no_satisfechos = crear_votantes(tamano_poblacion, 0.7, 0.4, "NOSAT")
poblacion = np.vstack((satisfechos, no_satisfechos))
poblacion = pd.DataFrame(poblacion)
poblacion.columns = ['opinion', 'responderia_encuesta']
poblacion['responderia_encuesta'] = pd.to_numeric(poblacion['responderia_encuesta'])

tamano_muestra = 250
muestra = poblacion.sample(tamano_muestra)
muestra_respondieron = muestra[muestra['responderia_encuesta'] == 1]
muestra_respondieron.groupby('opinion')['responderia_encuesta'].count()*100\
/len(muestra_respondieron)

**¿Qué podríamos concluir?**